# DQN implementation by fxd
#using atari games as a test

In [6]:
#DQN implementation by waterhorse
#based on MorvanZhou's reinforcement code
#using atari games as a test
import tensorflow as tf
import numpy as np
import random

np.random.seed(3)
tf.set_random_seed(3)
class DQN:
    def __init__(
        self,
        actions_n,
        features_n,
        learning_rate,
        batch_size,
        lamda,
        start_epsilon,
        final_epsilon
        replace_iter,
        memory_size,
    ):
        self.actions = actions_n#dimension for action space
        self.features_n =features_n#size for the input image 
        self.lr = rate
        self.start_epsilon = start_epsilon
        self.final_epsilon = final_epsilon
        self.epsilon = start_epsilon
        self.batch_size = batch_size
        self.lamda = lamda#decay
        self.replace = replace_iter
        self.__build__network()
        self.sess = tf.Session()
        self.saver = tf.train.Saver()
        self.sess.run(tf.global_variables_initializer())
        self.memory=[]
        self.memory_size = memory_size
        self.memory_counter = 0
        self.episode = 0
        
        target_net_params = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='target_net')
        evaluate_net_params = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='eval_net')

        with tf.variable_scope('hard_replacement'):
            self.target_replace_op = [tf.assign(t, e) for t, e in zip(t_params, e_params)]
        
        
        self.train_episode = 0#record 
        tf.summary.FileWriter("./log/",sess.graph)
        
        
    def _build_net_all(self):
        with tf.variable_scope('evaluate_network'):
            self.s = tf.placeholder(tf.float32,[None,64,64,4],name="state")
            self.a = tf.placeholder(tf.int32,[None,],name="action")
            self.r = tf.placeholder(tf.float32,[None,],name="reward")
            self.done = tf.placeholder(tf.int32,[None,],name="done")
            self.size = tf.shape(self.a)[0]#size of batch
            self.eval=self.build_net(s)
            
        with tf.variable_scope('target_network'):
            self.s_ = tf.placeholder(tf.float32,[None,64,64,4],name="state")
            self.target = self.build_net(s_)
            
        with tf.variable_scope('loss'):
            tar = tf.cond(self.done < 0.5,lambda:self.r + self.lamda*tf.reduce_max(self.target,axis = 1),
                         lambda:self.r)
            q_tar = tf.stop_gradient(tar)
            index = tf.stack([tf.range(1,self.size,dtype=tf.int32),self.a],axis=1)
            q_eval = tf.gather_nd(self.eval,index)
            self.loss = tf.reduce_mean(tf.squared_difference(self.q_tar, self.q_eval, name='TD'))
                                       
        with tf.variable_scope('train'):
            self._train_op = tf.train.AdamOptimizer(self.lr).minimize(self.loss)

            
    def build_net(self,s): 
        W_conv1 = weight_variable([8, 8, 4, 32])
        b_conv1 = bias_variable([32])

        W_conv2 = weight_variable([4, 4, 32, 64])
        b_conv2 = bias_variable([64])

        W_conv3 = weight_variable([3, 3, 64, 64])
        b_conv3 = bias_variable([64])
        
        h_conv1 = tf.nn.relu(conv2d(s, W_conv1, 4) + b_conv1)
        h_pool1 = max_pool_2x2(h_conv1)

        h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2, 2) + b_conv2)
        h_pool2 = max_pool_2x2(h_conv2)

        h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3, 1) + b_conv3)
        h_pool3 = max_pool_2x2(h_conv3)
        
        flat = tf.reshape(h_pool3,[self.size,-1])
        
        l1 = tf.layers.dense(flat,256,tf.nn.relu)
        l2 = tf.layers.dense(l1,64,tf.nn.relu)
        l3 = tf.layers.dense(l2,self.actions)
        
        return l3
                                    
        
    def weight_variable(shape):
        initial = tf.truncated_normal(shape, stddev = 0.01)
        return tf.Variable(initial)

    def bias_variable(shape):
        initial = tf.constant(0.01, shape = shape)
        return tf.Variable(initial)
    
    def conv2d(x, W, stride):
        return tf.nn.conv2d(x, W, strides = [1, stride, stride, 1], padding = "SAME")

    def max_pool_2x2(x):
        return tf.nn.max_pool(x, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = "SAME")
                                       
    def choose_action(self,s1):
        s1 = s1[np.newaxis,:]
        if(np.random.rand()>self.epsilon):
            action = np.random.randint(0,self.actions)
        else:
            action = self.sess.run(self.eval,feed_dict={self.s:s1})
            action = np.argmax(action)
        return action
                                           
    def save_net(self,name):
        self.saver.save(self.sess, "model/"+name)
        
    def load_net(self):
        load = tf.train.import_meta_graph('model/my-model-290.meta')
        load.restore(self.sess, tf.train.latest_checkpoint("model/"))
        
    def learn(self):
        if(self.episode%self.replace == 0):
            self.episode = 0
            self.sess.run(self.target_replace_op)
            print('replace target_network params')
            
        minibatch = random.sample(self.memory,self.batch_size)
        _ = self.sess.run([self._train_op],
                          feed_dict=
                          {
                            self.s : [d[0] for d in minibatch]
                            self.a : [d[1] for d in minibatch]
                            self.r : [d[2] for d in minibatch]
                            self.s_ : [d[3] for d in minibatch]
                            self.done : [d[4] for d in minibatch]
                          }
                         )
        self.episode += 1
        
        if(self.epsilon < self.final_epsilon):
            self.epsilon += (self.start_epsilon -self.final_epsilon) / 150000
        else:
            self.epsilon = self.final_epsilon
            
    def store(self,s,a,r,s_,done):
        if done:
            done = 1
        else:
            done = 0
            
        mem = [s,a,r,s_,done]
        if self.memory_size > len(self.memory):
            self.memory_size.append(mem)
        else:
            self.memory_counter = self.memory_counter % self.memory_size
            self.memory[self.memory_counter] = mem
        self.memory_counter +=1
        
                                       